# Create VOLOv7 for TF-Lite

You  Only  Look  Once

You know the drill: Create a virtual environment and install the requirements.

In [ ]:
# Library imports
import os
import random

# Project imports
import cv2
import numpy as np
import tensorflow as tf
import onnxruntime as ort
import matplotlib.pyplot as plt

### Getting YOLO

We'll start off by cloning YOLOv7 from the official repo.  We'll also get the latest weights.

YOLOv7 is written using pytorch.  Thankfully, those good folks at YOLO gave us an export script to translate their inference model to ONNX format, so we'll do that now.  All this happens in the next cell.  Sit tight.

### `export.py` Parameter explanation

- **grid**:  The grid parameter is an option allowing the export of the detection layer grid.
- **end2end**:  It is the option that allows the export of end-to-end ONNX graph which does both bounding box prediction and NMS.
- **simplify**:  It is the option by which we can select whether we want to simplify the ONNX graph using reparameterization.
- **topk-all**:  It's the option to select the top k object per image using IOU and confidence threshold.
- **iou-thres**:  It is the option to set the IOU threshold for NMS.
- **conf-thres**:  It is the option to select the confidence threshold score.
- **img_size and max-wh**:  These parameters are related to the size of the input image.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
!python3 export.py --weights yolov7.pt --grid --end2end --simplify --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640
%cd ..

Tada!  We have YOLO, and it's in ONNX format for us.  Here's what we get out of the box.

In [ ]:
# Order matters for cool freebies.
names = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush')

## Functions

In [ ]:
def letterbox(input_img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scale_up=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints

    old_shape = input_img.shape[:2] # current shape [height, width]

    # if it's just an int, assume it's a square
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    ratio = min(new_shape[0] / old_shape[0], new_shape[1] / old_shape[1])
    if not scale_up:  # only scale down, do not scale up (for better val mAP!)
        ratio = min(ratio, 1.0)

    # Compute padding, HxW
    hw_padding: tuple = (int(round(old_shape[1] * ratio)), int(round(old_shape[0] * ratio)))
    delta_w, delta_h = new_shape[1] - hw_padding[0], new_shape[0] - hw_padding[1]

    # minimum rectangle
    if auto:
        delta_h, delta_w = np.mod(delta_h, stride), np.mod(delta_w, stride)

    # divide padding into 2 sides
    delta_w /= 2
    delta_h /= 2

    # resize
    if old_shape[::-1] != hw_padding:
        output_img = cv2.resize(input_img, hw_padding, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(delta_h - 0.1)), int(round(delta_h + 0.1))
    left, right = int(round(delta_w - 0.1)), int(round(delta_w + 0.1))
    output_img = cv2.copyMakeBorder(output_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return output_img, ratio, (delta_w, delta_h)

# Notice we are using enumerate(output_data)
def display(frame, scale_deltas, ratio, colors, outputs, names=names):

    for (_, x0, y0, x1, y1, cls_id, score) in outputs:
        bound_box = np.array([x0, y0, x1, y1])
        bound_box -= np.array(scale_deltas * 2)
        bound_box /= ratio
        bound_box = bound_box.round().astype(np.int32).tolist()
        cls_id = int(cls_id)
        score = round(float(score), 3)
        score = round(score * 100)
        name = names[cls_id]
        color = colors[name]
        label = f'{name} {score}%'
        cv2.rectangle(img=frame,
                      pt1=bound_box[:2],
                      pt2=bound_box[2:],
                      color=color,
                      thickness=2)

        # We do it backwards. get size/coordinates before *actually* create the text
        (w, h), _ = cv2.getTextSize(text=label,
                                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                    fontScale=0.6,
                                    thickness=1)
        # class label
        cv2.rectangle(img=frame,
                      pt1=(bound_box[0]-1, bound_box[1]),
                      pt2=(bound_box[0]+w, bound_box[1]-h-6),
                      color=color,
                      thickness=-1)

        cv2.putText(img=frame,
            text=label,
            org=(bound_box[0]+5, bound_box[1] - 5),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=.5,
            color=[255, 255, 255],
            thickness=1,
            lineType=cv2.LINE_AA)

        cv2.imshow('VIDEO', frame)

## Common Variables

In [ ]:
cuda = False
os_path = os.getcwd()

#Loading image(s) for testing.
img = cv2.imread(f'{os_path}/content/6.jpg')

#Create some random colors for bounding box visualizations.
colors = {name:[random.randint(0, 255) for _ in range(3)] for i,name in enumerate(names)}

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Pre-processing the image.
image = img.copy()
image, ratio, scale_deltas = letterbox(image, auto=False)
image = image.transpose((2, 0, 1))
image = np.expand_dims(image, 0)
image = np.ascontiguousarray(image)

# good ol' normalization
np_img = image.astype(np.float32)
np_img /= 255
print(f'shape: {np_img.shape}')

### Run inference with ONNX

In [ ]:
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']
session = ort.InferenceSession(f'{os_path}/yolov7/yolov7.onnx', providers=providers)

#Getting onnx graph input and output names.
output_names = [i.name for i in session.get_outputs()]
input_names = [i.name for i in session.get_inputs()]
input_feed = {input_names[0]: np_img}
onnx_outputs = session.run(output_names=output_names, input_feed=input_feed)[0]
ori_img = [img.copy()]

#Visualizing bounding box prediction.
display(frame=image, scale_deltas=scale_deltas, ratio=ratio, colors=colors, outputs=onnx_outputs)

In [ ]:
# Sometimes the command needs to be ran in your shell, dunno why ATM.
!onnx-tf convert -i yolov7/yolov7.onnx -o .

## Convert model to TF-lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(os_path)
tflite_model = converter.convert()

with open(f'{os_path}/yolov7_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="yolov7_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
interpreter.set_tensor(input_details[0]['index'], np_img)

interpreter.invoke()

tfl_out = interpreter.get_tensor(output_details[0]['index'])
print(f'shape: {tfl_out.shape}')

### Run inference on TF-Lite

In [ ]:

display(frame=image, scale_deltas=scale_deltas, ratio=ratio, colors=colors, outputs=tfl_out)